In [1]:
import sys
sys.path.append('./src')
from pipeline import GCLPipeline

import os.path as osp
import json
import pandas as pd

import torch
import torch_geometric.transforms as T

from torch.optim import Adam

from tqdm import tqdm 

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

DATA_PATH = "./data"
BENCHMARK_PATH = "./benchmark"

In [2]:
STRATEGIES = json.load(open(osp.join(BENCHMARK_PATH, "benchmark.json")))
CONFIG = STRATEGIES["config"]
TRY_GPU = CONFIG["try_gpu"]
DEVICE = torch.device("cuda" if torch.cuda.is_available() and TRY_GPU else "cpu")

In [3]:
outputs = []

for strategy in STRATEGIES["strategies"]:

    print("\n")

    # Initialization
    pipeline = GCLPipeline.from_strategy(strategy, DEVICE)

    # Data
    dataset, num_features = GCLPipeline.init_dataset(
        strategy["dataset"], DATA_PATH, T.NormalizeFeatures(), strategy["batch_size"]
    )
    
    # Encoder
    encoder_params = strategy["encoder_params"]
    encoder_params["input_dim"] = num_features
    encoder_model = pipeline.init_encoder(encoder_params, DEVICE)

    # Training
    train_params = strategy["train_params"]
    lr = train_params["learning_rate"]
    epochs = train_params["epochs"]

    optimizer = Adam(encoder_model.parameters(), lr=lr)
    losses = []

    with tqdm(total=epochs, desc='(T)') as pbar:
        for epoch in tqdm(range(1, epochs+1)):
            loss = pipeline.train_epoch(
                encoder_model, dataset, optimizer, DEVICE
            )
            pbar.set_postfix({'loss': loss})

            pbar.update()

    test_result = pipeline.test(encoder_model, dataset, DEVICE)
    print(f'(E): Best test F1Mi={test_result["micro_f1"]:.4f}, F1Ma={test_result["macro_f1"]:.4f}')

    outputs.append({
        "Data": strategy["dataset"],
        "Method": strategy["method"],
        "Archi": strategy["architecture"],
        "Mode": strategy["mode"],
        "Obj": strategy["objective"],
        "Neg": strategy["negative"],

        "Aug1": strategy["augmentation1"],
        "Aug1Strat": strategy["augmentation1_strat"], 
        "Aug2": strategy["negative"],
        "Aug2Strat": strategy["negative"], 
        "MicroF1": test_result["micro_f1"],
        "MacroF1": test_result["macro_f1"]
    })





##### InfoGraph #####
Dataset initialization
	 # features: 18
Encoder initialization
	 input dim: 18
	 hidden dim: 32
	 # layers: 2
	 projection dim: None
	 activation: <class 'torch.nn.modules.activation.ReLU'>


(T): 100%|██████████| 100/100 [00:10<00:00,  9.20it/s, loss=-2.24]
c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number 

(E): Best test F1Mi=0.5714, F1Ma=0.5478


##### TransductiveDGI #####
Dataset initialization
	 # features: 1433
Encoder initialization
	 input dim: 1433
	 hidden dim: 512
	 # layers: 2
	 projection dim: None
	 activation: None


(LR): 100%|██████████| 5000/5000 [00:15<00:00, best test F1Mi=0.82, F1Ma=0.772] 


(E): Best test F1Mi=0.8199, F1Ma=0.7723


##### GRACE #####
Dataset initialization
	 # features: 1433
Encoder initialization
	 input dim: 1433
	 hidden dim: 32
	 # layers: 2
	 projection dim: 32
	 activation: <class 'torch.nn.modules.activation.ReLU'>


(T):   0%|          | 0/1000 [00:00<?, ?it/s]c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)
(LR): 100%|██████████| 5000/5000 [00:14<00:00, best test F1Mi=0.79, F1Ma=0.773] 


(E): Best test F1Mi=0.7904, F1Ma=0.7726


##### GRACE #####
Dataset initialization
	 # features: 1433
Encoder initialization
	 input dim: 1433
	 hidden dim: 32
	 # layers: 2
	 projection dim: 32
	 activation: <class 'torch.nn.modules.activation.ReLU'>


(T):   0%|          | 0/1000 [00:00<?, ?it/s]c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)
(T):   0%|          | 0/1000 [00:00<?, ?it/s]


AssertionError: 

In [ ]:
output_df = pd.DataFrame(outputs)
output_df

,Data,Method,Archi,Mode,Obj,Neg,Aug1,Aug1Strat,Aug2,Aug2Strat,MicroF1,MacroF1
0,PTC_MR,InfoGraph,SingleBranch,G2L,JSD,None,None,None,None,None,0.428571,0.416667
1,Cora,TransductiveDGI,SingleBranch,G2L,JSD,None,None,Compose,None,None,0.808824,0.780598
2,Cora,GRACE,DualBranch,L2L,InfoNCE,None,"[EdgeRemoving, FeatureMasking]",Compose,None,None,0.735294,0.714116
